In [1]:
import numpy as np
import random
import pandas as pd
import arff
from sklearn import preprocessing
 

In [6]:
matrix = [
    [5,2,3],
    [4,6,6],
    [7,8,3],
    [10,11,12]
]
y = np.array([1,0,0,1])
df = pd.DataFrame(matrix, columns=list('xyz'))


In [7]:
y

array([1, 0, 0, 1])

In [309]:
df = df.apply(lambda df : (df - np.mean(df)) / np.std(df, ddof = 1))
df

,x,y,z
0,-0.566947,-1.258306,-0.707107
1,-0.944911,-0.198680,0.000000
2,0.188982,0.331133,-0.707107
3,1.322876,1.125853,1.414214


In [311]:
df.columns = np.arange(0,df.shape[1],1)
df
N = len(y)
init_value = np.array([-0.7, 1.1, 1.1, -0.9])
# np.random.normal(0,1,df.shape[1]) # mu, sigma, n np.array([-0.6973182, 1.1349651, 1.1119318, -0.8707776])
gamma = 0.5 * 0.01
update_beta = beta = init_value
update_beta = update_beta + 1
times = 0

In [12]:
def glm_st(x, y, alpha=0.5, lam=0.01, epsilon=0.00001):
    x = x.apply(lambda x : (x - np.mean(x)) / np.std(x, ddof = 1))
    x.insert(0,'截距',np.zeros(len(x)) + 1)
    x.columns = np.arange(0,x.shape[1],1) #rename
    
    N = len(y)
    init_value = np.random.normal(0,1,x.shape[1]) # mu, sigma, n np.array([-0.6973182, 1.1349651, 1.1119318, -0.8707776])
    gamma = alpha * lam
    update_beta = beta = init_value
    update_beta = update_beta + 1
    times = 0
    while(np.linalg.norm(update_beta - beta) > epsilon):
        update_beta = beta
        times = times + 1
        if times > 100 :
            break
        else:
            for index in range(len(init_value)):
                a1 = x[index] @ y / N
                a2 = (x.T @ x[index].T) @ beta / N
                z = a1 - a2 + beta[index]
                if gamma >= abs(z):
                    condition = 0
                else:
                    if z > 0:
                        condition = z - gamma
                    else:
                        condition = z + gamma
                beta[index] = condition / (1 + lam - gamma)
    return(beta)
                
                

In [14]:
glm_st(df, y,alpha = 1)

array([ 0.49      ,  0.94643293, -0.63740868, -0.10504863])

In [15]:
def glm_st(x, y, alpha=0.5, lam=0.01, epsilon=0.00001):
    xnorm = x.apply(lambda x : (x - np.mean(x)) / np.std(x, ddof = 1))
    xnorm.insert(0,'截距',np.ones(len(x)))
    xnorm = xnorm.to_numpy()
    ynew = y.to_numpy()
    ynew.reshape(1,-1)
    
    N = len(y)
    init_value = np.random.normal(0,1,[1,xnorm.shape[1]]) # mu, sigma, n np.array([-0.6973182, 1.1349651, 1.1119318, -0.8707776])
    
    gamma = alpha * lam
    update_beta = beta = init_value
    update_beta = update_beta + 1
    times = 0
    while(np.linalg.norm(update_beta - beta) > epsilon):
        update_beta = beta
        times = times + 1
        if times > 100 :
            break
        else:
            for index in range(len(init_value)):
                a1 = xnorm[:,index:index+1].T @ ynew / N
                #a2 = (((xnorm.T).dot(xnorm[:,index:index+1]))).T.dot(beta.T) / N
                a2 = (xnorm.T @ xnorm[:,index:index+1]).T @ beta.T
                
                z = a1 - a2 + beta[0,index]
                if gamma >= abs(z):
                    condition = 0
                else:
                    if z > 0:
                        condition = z - gamma
                    else:
                        condition = z + gamma
                beta[0,index] = condition / (1 + lam - gamma)
    return(beta)
                
                